In [1]:
# !pip install gingerit
# !pip install contractions

In [30]:
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

warnings.filterwarnings('ignore')

import os
import math
import pandas as pd
import Embedding as embd
import numpy as np
from keras.preprocessing.text import Tokenizer # https://keras-cn.readthedocs.io/en/latest/preprocessing/text/
from keras.utils import to_categorical
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import nltk as nl
nl.download('punkt')

import re

import contractions
from sklearn.model_selection import train_test_split 
from gingerit.gingerit import GingerIt


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\13037\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
NN=30000
data_train=pd.read_csv('../dataset/Train.csv')
data_train=data_train[0:NN]
X_train=data_train['TEXT'].values
Y_train_1D=data_train['Label'].values
Y_train=to_categorical(Y_train_1D)

data_test=pd.read_csv('../dataset/Test.csv')
data_test=data_test[0:NN]
X_test=data_test['TEXT'].values


In [32]:
X_rm=embd.corpus_pre(X_train)

In [33]:
cleanedSentences = []
parser = GingerIt() #uesd for corrcect misspelling
regex = re.compile('[^a-zA-z]')#uesd for deleting non-alphabet characters
for sentence in X_rm:
    URLless_string = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', sentence)
    contfix = contractions.fix(URLless_string)
    AlphabetOnly = regex.sub(' ', contfix)
    onespace=' '.join(AlphabetOnly.split())
    cleanedSentences.append(onespace)


In [34]:
nn=100
x_train=cleanedSentences[nn:]
x_test=cleanedSentences[0:nn]
y_train=Y_train_1D[nn:]
y_test=Y_train_1D[0:nn]

In [35]:
# 用TF-IDF计算单词的权重
from nltk.corpus import stopwords 
stopper=set(stopwords.words('english'))
tf = TfidfVectorizer(stop_words = stopper, max_df = 0.9)

train_features = tf.fit_transform(x_train)
print(len(tf.get_feature_names()))
test_features = tf.transform(x_test)

40159


In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

clf = GaussianNB().fit(train_features.toarray(), y_train)
pred_labels = clf.predict(test_features.toarray())

# accuracy
print('accuracy: ', metrics.accuracy_score(y_test, pred_labels))

accuracy:  0.15
